In [3]:
import sys
import os
sys.path.append("/home/pi/EE123/Chirp-EE123")

cur_path = os.path.dirname('/home/pi/EE123/Chirp-EE123')
sys.path.insert(0, '/home/pi/EE123/Chirp-EE123/src')

from utils.EE123TNC import *
from utils.hamming import *

import sounddevice as sd
import RPi.GPIO as GPIO

# Load image/gif into queue

In [2]:
callsign = "KM6QHM"
Digi =b'WIDE1-1,WIDE2-1'
dest = "APCAL"
fname = b"../../asset/calBlue.tiff"
f = open(fname,"rb")
fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 10)
print("Putting packets in Queue")

starttime = time.time()
npack = 0
prefix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(40,)).tolist())
Qout=Queue.Queue()
while True:
    chunk=f.read(256)
    if chunk:
        info = chunk
        packet = ax25.UI(
            destination=dest,
            source=callsign,
            info=info,
            digipeaters=Digi.split(b','),
            )
        bits=modem.NRZ2NRZI(prefix+packet.unparse()+prefix)
        packets = modem.modulate(bits)
        Qout.put(packets)
    else:
        break

print("Processing took:", time.time()-starttime)
print("Done")

f.close()

Putting packets in Queue
Processing took: 3.018162727355957
Done


# Transmit image/gif

In [14]:
GPIO.cleanup()
GPIO.setmode(GPIO.BOARD)
PTT = 16
GPIO.setup(PTT, GPIO.OUT, initial = 0)

print("Starting transmission...")
starttime = time.time()

GPIO.output(PTT, GPIO.HIGH)
while(not(Qout.empty())):
    sig=Qout.get()
    sd.play(sig*0.2,samplerate=48000,device=usb_idx, blocking=True)
GPIO.output(PTT, GPIO.LOW)

print("Done")
print(time.time() - starttime)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  after removing the cwd from sys.path.


Starting transmission...
Done
115.83593916893005
